# wav2vec2-base-960h no LM

In [1]:
import utils

## 0. Download LibriSpeech test-clean data

In [2]:
# from bucket
!gsutil -m -q cp -n -r gs://capstone_datasets/* ./datasets/

In [3]:
# load extracted lr data as dataset
librispeech_eval = utils.load_dataset('datasets/librispeech/test', "clean", split="test")

Resolving data files:   0%|          | 0/2709 [00:00<?, ?it/s]

Using custom data configuration test-8e580f273879a10c


Extracting data files:   0%|          | 0/89 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset audiofolder downloaded and prepared to /home/antonin/.cache/huggingface/datasets/audiofolder/test-8e580f273879a10c/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc. Subsequent calls will reuse this data.


## 1. Map each audio file to its ground truth

In [4]:
librispeech_eval = librispeech_eval.map(utils.map_to_ground_truth)

  0%|          | 0/2620 [00:00<?, ?ex/s]

## 2. Load Wav2Vec 2.0 model and tokenizer

In [5]:
tokenizer, model = utils.load_wav2vec_model("facebook/wav2vec2-base-960h")

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/opt/conda/lib/python3.7/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:757: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,


Downloading:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 3. Compute prediction

In [6]:
result = librispeech_eval.map(utils.map_to_pred, fn_kwargs={"model": model, "tokenizer": tokenizer})

  0%|          | 0/2620 [00:00<?, ?ex/s]

## 4. Compute metric

In [7]:
print('WER with wav2vec2-base-960h on lr-test-clean:', round(100 * utils.wer(result["ground_truth"], result["transcription"]), 1), '%.')

WER with wav2vec2-base-960h on lr-test-clean: 3.4 %.


We get the same metric as in the original paper https://arxiv.org/pdf/2006.11477.pdf page 15.

## 5. Save dataset and prediction to disk

In [8]:
result.save_to_disk(utils.os.path.join(utils.predictions_path, 'lr-clean-test-w2v2-base-960h.hf'))

In [9]:
result

Dataset({
    features: ['audio', 'label', 'ground_truth', 'logits', 'transcription'],
    num_rows: 2620
})

## 6. Load dataset

In [10]:
dataset = utils.load_from_disk(utils.os.path.join(utils.predictions_path, 'lr-clean-test-w2v2-base-960h.hf'))

In [11]:
dataset

Dataset({
    features: ['audio', 'label', 'ground_truth', 'logits', 'transcription'],
    num_rows: 2620
})

## 7. Send to bucket

In [12]:
!gsutil -m cp -n -r ./predictions/ gs://capstone_datasets/librispeech/test/ 

Skipping existing item: gs://capstone_datasets/librispeech/test/predictions/lr-clean-test-w2v2-base-960h.hf/dataset_info.json
Skipping existing item: gs://capstone_datasets/librispeech/test/predictions/lr-clean-test-w2v2-base-960h.hf/dataset.arrow
Skipping existing item: gs://capstone_datasets/librispeech/test/predictions/lr-clean-test-w2v2-base-960h.hf/state.json


## 8. Copy from bucket and load

In [13]:
!gsutil -m cp -n -r gs://capstone_datasets/librispeech/test/predictions/* ./predictions/

Skipping existing item: file://./predictions/lr-clean-test-w2v2-base-960h.hf/dataset.arrow
Skipping existing item: file://./predictions/lr-clean-test-w2v2-base-960h.hf/dataset_info.json
Skipping existing item: file://./predictions/lr-clean-test-w2v2-base-960h.hf/state.json


In [14]:
dataset = utils.load_from_disk(utils.os.path.join(utils.predictions_path, 'lr-clean-test-w2v2-base-960h.hf'))

In [15]:
dataset

Dataset({
    features: ['audio', 'label', 'ground_truth', 'logits', 'transcription'],
    num_rows: 2620
})